# Дипломный проект

## Модель прогнозирования стоимости жилья для агентства недвижимости

### 3. Этап. Создание модели

In [27]:
import random
import numpy as np 
import pandas as pd 
import sys
import optuna

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import KFold
from sklearn import metrics
from tqdm.notebook import tqdm
from category_encoders import TargetEncoder, CatBoostEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

Cделаем следующее:

* Обработаем и отнормируем признаки
* Построим "наивную" модель, предсказывающую цену по общей площади и городу (с ней будем сравнивать другие модели)
* Построим модель при помощи LinearRegression
* Обучим модель на основе случайного леса RandomForestRegressor
* Обучим модель с L1 и L2 регуляризаций ElasticNetCV
* На основе предыдущих шагов выберем оптимальную модель

In [28]:
import matplotlib.pyplot as plt
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline
#отключим оповещения
import warnings
warnings.filterwarnings("ignore")

In [29]:
# зафиксируем RANDOM_SEED, чтобы эксперименты были воспроизводимы
RANDOM_SEED = 42
TEST_SIZE = 0.2

In [30]:
df = pd.read_csv('data/data_model_diplom.csv')
display(df.head(3))
df.info()

,status,baths,city,sqft,zipcode,state,target,pool_encoded,Type,Year built,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating _mean,school_dist_min
0,Active,4.0,Southern Pines,2900,28387,NC,418000,False,single_family_home,2019,True,False,False,True,5.2,2.70
1,For Sale,3.0,Spokane Valley,1947,99216,WA,310000,False,single_family_home,2019,False,False,False,False,4.0,1.01
2,Active,2.0,Mason,3588,50401,IA,244900,False,single_family_home,1970,True,True,False,False,3.8,5.60


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210669 entries, 0 to 210668
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   status               210669 non-null  object 
 1   baths                210669 non-null  float64
 2   city                 210669 non-null  object 
 3   sqft                 210669 non-null  int64  
 4   zipcode              210669 non-null  int64  
 5   state                210669 non-null  object 
 6   target               210669 non-null  int64  
 7   pool_encoded         210669 non-null  bool   
 8   Type                 210669 non-null  object 
 9   Year built           210669 non-null  object 
 10  Heating_encoded      210669 non-null  bool   
 11  Cooling_encoded      210669 non-null  bool   
 12  Parking_encoded      210669 non-null  bool   
 13  fireplace_encoded    210669 non-null  bool   
 14  school_rating _mean  210669 non-null  float64
 15  school_dist_min  

In [31]:
# Составим список булевых признаков:
bin_features = ['pool_encoded','Heating_encoded','Cooling_encoded','Parking_encoded','fireplace_encoded']

# Составим список категориальных признаков:
cat_features = ['status','city','zipcode','state','Type','Year built']
 
# Составим список числовых признаков:
num_features = ['baths', 'sqft', 'target', 'school_rating _mean', 'school_dist_min']

In [32]:
# подсчет количества уникальных значений в каждой категориальной колонке
for col in cat_features:
   unique_values = df[col].nunique()
   print(f"Количество уникальных значений в категориальной колонке {col}: {unique_values}")

Количество уникальных значений в категориальной колонке status: 12
Количество уникальных значений в категориальной колонке city: 1522
Количество уникальных значений в категориальной колонке zipcode: 3962
Количество уникальных значений в категориальной колонке state: 34
Количество уникальных значений в категориальной колонке Type: 12
Количество уникальных значений в категориальной колонке Year built: 205


In [33]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    # переведем признак зип кода в категориальный
    df_output['zipcode'] = df_output['zipcode'].astype(str)
    # переведем признак год в категориальный
    df_output['Year built'] = df_output['Year built'].astype(str)
    # Нормализация и логорифмирование данных
    #scaler = MinMaxScaler()
    for column in ['baths', 'sqft', 'target', 'school_rating _mean', 'school_dist_min']:
        #df_output[column] = scaler.fit_transform(df_output[[column]])[:,0]
        # Логорифмирование
        df_output[column] = df_output[column].apply(lambda x: abs(x))
        constant = 1e-6
        df_output[column] = np.log(df_output[column] + constant)
        
    # ################### Categorical Features ############################################################## 
 
    ohe_status = OneHotEncoder(sparse=False)
    ohe_state = OneHotEncoder(sparse=False)
    ohe_Type = OneHotEncoder(sparse=False)

    status_ohe = ohe_status.fit_transform(df_output['status'].values.reshape(-1,1))
    state_ohe = ohe_state.fit_transform(df_output['state'].values.reshape(-1,1))
    Type_ohe = ohe_Type.fit_transform(df_output['Type'].values.reshape(-1,1))

    le = LabelEncoder()
    state_label = le.fit_transform(df_output['state'])

    year_le = LabelEncoder()
    year_ord = year_le.fit_transform(df_output['Year built'])

    city_le = LabelEncoder()
    city_label = city_le.fit_transform(df_output['city'])

    zip_le = LabelEncoder()
    zip_label = zip_le.fit_transform(df_output['zipcode'])

    # Adding encoded categorical features to the output dataframe
    df_output = df_output.join(pd.DataFrame(status_ohe, columns=['status_' + str(cat) for cat in ohe_status.categories_[0]]))
    df_output = df_output.join(pd.DataFrame(state_ohe, columns=['state_' + str(cat) for cat in ohe_state.categories_[0]]))
    df_output = df_output.join(pd.DataFrame(Type_ohe, columns=['Type_' + str(cat) for cat in ohe_Type.categories_[0]]))
    df_output['state_label'] = state_label
    df_output['year_ord'] = year_ord
    df_output['city_label'] = city_label
    df_output['zip_label'] = zip_label

    # Dropping original categorical columns
    df_output.drop(['status', 'state', 'Type', 'city', 'zipcode','Year built'], axis=1, inplace=True)
    
    return df_output

In [34]:
# Запускаем и проверяем, что получилось
df_encoded = preproc_data(df)
df_encoded.sample(10)

,baths,sqft,target,pool_encoded,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating _mean,school_dist_min,...,Type_modern,Type_multi_family_home,Type_other,Type_ranch,Type_single_family_home,Type_townhouse,state_label,year_ord,city_label,zip_label
81347,0.693148,7.244228,11.277203,False,True,False,True,False,1.098613,-0.301104,...,0.0,0.0,0.0,0.0,1.0,0.0,8,72,242,2072
128246,1.098613,7.875879,13.384574,True,True,True,True,False,1.704748,0.963175,...,0.0,0.0,0.0,0.0,1.0,0.0,5,180,204,1229
146360,0.693148,7.845416,12.425208,True,True,True,True,True,1.945910,0.058270,...,0.0,0.0,0.0,0.0,1.0,0.0,28,163,612,2537
167718,0.693148,7.539027,12.834681,False,True,True,True,True,1.791760,0.173954,...,0.0,0.0,0.0,0.0,1.0,0.0,20,169,727,3078
201661,1.098613,7.441320,12.417144,False,True,True,True,False,1.609438,0.322084,...,0.0,0.0,0.0,0.0,1.0,0.0,28,193,61,2745
97621,0.693148,7.526718,12.400405,True,True,True,False,False,1.667707,-0.094310,...,0.0,0.0,0.0,0.0,1.0,0.0,5,190,644,928
162567,0.693148,7.511525,12.685377,False,False,False,False,False,1.308333,-2.302575,...,0.0,0.0,0.0,0.0,1.0,0.0,28,204,608,2478
72308,0.693148,6.984716,12.345835,True,True,True,False,False,1.740466,0.231113,...,0.0,0.0,0.0,0.0,0.0,0.0,5,185,856,1034
78554,0.693148,7.732808,12.089539,False,True,True,True,False,1.193923,-1.427112,...,0.0,0.0,0.0,0.0,1.0,0.0,28,184,681,2422
105703,1.609438,8.172447,11.511925,False,True,True,True,False,0.693148,2.379546,...,0.0,1.0,0.0,0.0,0.0,0.0,22,106,263,1576


In [35]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210669 entries, 0 to 210668
Data columns (total 72 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   baths                    210669 non-null  float64
 1   sqft                     210669 non-null  float64
 2   target                   210669 non-null  float64
 3   pool_encoded             210669 non-null  bool   
 4   Heating_encoded          210669 non-null  bool   
 5   Cooling_encoded          210669 non-null  bool   
 6   Parking_encoded          210669 non-null  bool   
 7   fireplace_encoded        210669 non-null  bool   
 8   school_rating _mean      210669 non-null  float64
 9   school_dist_min          210669 non-null  float64
 10  status_Active            210669 non-null  float64
 11  status_Auction           210669 non-null  float64
 12  status_Back on Market    210669 non-null  float64
 13  status_Coming Soon       210669 non-null  float64
 14  stat

In [36]:
# Разделим датасет

y = df_encoded.target.values
X = df_encoded.drop(['target'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, shuffle=True, random_state=RANDOM_SEED)

## Обучение модели

### Model 1: Создадим "наивную" модель

Эта модель будет предсказывать среднюю стоимость по общей площади и городу. C ней будем сравнивать другие модели.

In [37]:
# Наивная модель
class NaiveModel:
    def __init__(self):
        self.means = None

    def fit(self, X, y):
        X_df = pd.DataFrame(X, columns=['city_label'])
        y_df = pd.DataFrame(y, columns=['target'])
        df = pd.concat([X_df, y_df], axis=1)
        self.means = df.groupby(['city_label'])['target'].mean().reset_index()

    def predict(self, X):
        X = pd.DataFrame(X, columns=['city_label']).copy()
        X['mean'] = np.nan
        for idx, row in self.means.iterrows():
            X.loc[(X['city_label'] == row['city_label']), 'mean'] = row['target']
        
        X['mean'].fillna(X['mean'].mean(), inplace=True)
        return X['mean'].to_numpy()

naive_model = NaiveModel()
naive_model.fit(X_train, y_train)
y_pred_train = naive_model.predict(X_train)
y_pred_test = naive_model.predict(X_test)

mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)
mae_train = mean_absolute_error(y_train, y_pred_train)
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")

Train MSE: 0.43
Test MSE: 0.43
Train MAE: 0.48
Test MAE: 0.48
Train R2: -0.00
Test R2: -0.00


MAE (Средняя абсолютная ошибка, Mean Absolute Error) - это мера ошибки, вычисленная как среднее значение абсолютных значений ошибок. Меньшие значения MAE указывают на лучшую точность модели.

В данном случае:
* Train MAE: 0.48 - это средняя абсолютная ошибка на обучающей выборке.
* Test MAE: 0.48 - это средняя абсолютная ошибка на тестовой выборке. 

R^2 (коэффициент детерминации) - статистическая мера, которая показывает, насколько хорошо вариации зависимой переменной объясняются моделью. Значения R^2 находятся в диапазоне от -∞ до 1. Чем ближе значение R^2 к 1, тем лучше модель объясняет зависимость между переменными. 

В данном случае:
* Train R2: -0.00 - это коэффициент детерминации на обучающей выборке.
* Test R2: -0.00 - это коэффициент детерминации на тестовой выборке. 

Проанализировав результаты, можно сделать следующие выводы:

Значения MSE и MAE, как для обучающей, так и для тестовой выборок, практически одинаковы. Это говорит о том, что модель демонстрирует схожую производительность на обеих выборках и не страдает от явного переобучения или недообучения.

Значения R^2 близки к нулю, что говорит о том, что модель слабо объясняет зависимость между переменными. Это может свидетельствовать о низкой предсказательной способности модели.

### Model 2: LinearRegression

In [38]:
# создаём модель линейной регрессии
model = LinearRegression(fit_intercept=False)

# вычисляем коэффициенты регрессии
model.fit(X_train, y_train)

# делаем предсказания с помощью модели
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# вычисляем требуемые метрики
mse_train = metrics.mean_squared_error(y_train, y_train_pred)
mse_test = metrics.mean_squared_error(y_test, y_test_pred)
mae_train = metrics.mean_absolute_error(y_train, y_train_pred)
mae_test = metrics.mean_absolute_error(y_test, y_test_pred)
r2_train = metrics.r2_score(y_train, y_train_pred)
r2_test = metrics.r2_score(y_test, y_test_pred)

# выводим метрики
print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")

Train MSE: 0.23
Test MSE: 0.23
Train MAE: 0.35
Test MAE: 0.34
Train R2: 0.45
Test R2: 0.45


Среднеквадратическая ошибка (Mean Squared Error, MSE) В данном случае, как для обучающей, так и для тестовой выборки MSE составляет 0.23. Низкие значения MSE указывают на более точные предсказания модели.

Средняя абсолютная ошибка (Mean Absolute Error, MAE) В данном случае MAE составляет 0.35 для обучающей и 0.34 для тестовой выборки. Низкие значения MAE также говорят о хорошей точности предсказания.

Коэффициент детерминации (R2) В данном случае R2 составляет 0.45 как для обучающей, так и для тестовой выборки, что означает, что модель объясняет 45% дисперсии зависимой переменной.

Результаты модели показывают, что она работает с одинаковой точностью на обучающем и тестовом наборе данных, что свидетельствует о том, что модель не переобучилась. Однако, коэффициент детерминации R2 указывает на то, что есть еще большой потенциал для улучшения модели, поскольку ходят возможны другие альтернативы, такие как увеличение размера выборки, использование других переменных или алгоритмов для еще большей предсказательной точности.

### Model 3: RandomForestRegressor

In [63]:
# Создаем экземпляр модели RandomForestRegressor
rf_regressor = RandomForestRegressor(random_state=RANDOM_SEED)

# Обучаем модель на обучающих данных
rf_regressor.fit(X_train, y_train)

# Предсказания на обучающих и тестовых данных
y_train_pred = rf_regressor.predict(X_train)
y_test_pred = rf_regressor.predict(X_test)

# Вычисляем метрики
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

# Выводим метрики
print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")

MemoryError: could not allocate 8388608 bytes

Среднеквадратическая ошибка (MSE) - В данном случае, значение MSE для обучающей выборки составляет 0.01, тогда как для тестовой выборки - 0.07. Такое различие может указывать на небольшое переобучение модели.

Средняя абсолютная ошибка (MAE) - В данном случае, значения MAE составили 0.06 для обучающей выборки и 0.16 для тестовой выборки. Разница между этими значениями также может указывать на переобучение.

Коэффициент детерминации (R2) - В данном случае, R2 равен 0.98 для обучающей выборки и 0.84 для тестовой выборки. Эти значения указывают на то, что модель довольно хорошо работает на обучающей выборке и неплохо предсказывает на тестовой выборке, хотя значение R2 на тестовой выборке заметно ниже, что также может указывать на переобучение модели.

В целом, результаты показывают, что модель имеет хорошую производительность на обучающей выборке, но на тестовой выборке результаты хуже, что может свидетельствовать о переобучении.

### Model 4: ElasticNetCV

In [40]:
# Создаем и тренируем модель ElasticNetCV с кросс-валидацией по 5 фолдам
model_el = ElasticNetCV(cv=5, random_state=RANDOM_SEED)
model_el.fit(X_train, y_train)

# Предсказания для обучающей и тестовой выборок
y_train_pred = model_el.predict(X_train)
y_test_pred = model_el.predict(X_test)

# MSE
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

# MAE
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

# R2
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

# Выводим метрики
print(f"Train MSE: {mse_train:.2f}")
print(f"Test MSE: {mse_test:.2f}")
print(f"Train MAE: {mae_train:.2f}")
print(f"Test MAE: {mae_test:.2f}")
print(f"Train R2: {r2_train:.2f}")
print(f"Test R2: {r2_test:.2f}")

Train MSE: 0.37
Test MSE: 0.38
Train MAE: 0.45
Test MAE: 0.45
Train R2: 0.12
Test R2: 0.11


MSE (Mean Squared Error) - В данном случае обучающая выборка имеет значение 0.37, а тестовая - 0.38. Эти числа близки, что означает, что модель не переобучилась, но в то же время обобщающая способность модели может быть не очень хорошей из-за высоких значений ошибок.

MAE (Mean Absolute Error) - В данном случае средняя абсолютная ошибка составляет 0.45 как для обучающей, так и для тестовой выборки. Это означает, что в среднем модель ошибается на 0.45 при предсказаниях.

R2 (коэффициент детерминации) - В данном случае R2 равен 0.12 для обучающей выборки и 0.11 для тестовой выборки. Эти значения невысоки, что говорит о плохом качестве модели и низкой способности объяснить изменчивость данных.

Вывод: по представленным результатам видно, что модель имеет среднюю ошибку и невысокий коэффициент детерминации. Это означает, что модель не является оптимальной.

### Что в итоге получаем:

In [41]:
data = {'Metric': ['Train MSE', 'Test MSE', 'Train MAE', 'Test MAE', 'Train R2', 'Test R2'],
        'NaiveModel': [0.43, 0.43, 0.48, 0.48, -0.00, -0.00],
        'LinearRegression': [0.23, 0.23, 0.35, 0.34, 0.45, 0.45],
        'RandomForestRegressor': [0.01, 0.07, 0.06, 0.16, 0.98, 0.84],
        'ElasticNetCV': [0.37, 0.38, 0.45, 0.45, 0.12, 0.11]}

df_metric = pd.DataFrame(data)
df_metric.head(6)

,Metric,NaiveModel,LinearRegression,RandomForestRegressor,ElasticNetCV
0,Train MSE,0.43,0.23,0.01,0.37
1,Test MSE,0.43,0.23,0.07,0.38
2,Train MAE,0.48,0.35,0.06,0.45
3,Test MAE,0.48,0.34,0.16,0.45
4,Train R2,-0.00,0.45,0.98,0.12
5,Test R2,-0.00,0.45,0.84,0.11


На основе этих результатов, RandomForestRegressor кажется наилучшей моделью, так как она имеет наименьший показатель ошибок MSE и MAE, вместе с высоким коэффициентом детерминации R2 (0.84) на тестовых данных.

In [51]:
import pickle

In [ ]:
# Сохранение выбранной обученной модели в файл pickle
with open("data/diplom_web/best_rf_regressor_model.pkl", "wb") as f:
    pickle.dump(rf_regressor, f)

### Далее:

* Подгрузим обученную модель rf_regressor
* Напишем функцию для предсказания стоимости недвижимости по входным параметрам

In [58]:
import numpy as np 
import pandas as pd 
import pickle
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Загрузка сохраненной модели из файла pickle
with open("data/diplom_web/best_rf_regressor_model.pkl", "rb") as f:
    loaded_model = pickle.load(f)

In [60]:
# создадим тестовый набор 
data = [
    ('Active', 3.0, 'Washington', 801, 20004, 'DC', False, 'other', 1991, False, False, False, False, 6.0, 0.2),
    ('Active', 2.0, 'Dallas', 832, 75219, 'TX', False, 'condo', 1998, False, True, False, False, 2.6, 0.6),
    ('Active', 4.0, 'Dallas', 2102, 75219, 'TX', False, 'condo', 2019, False, True, False, False, 2.0, 0.1),
    ('Active', 2.0, 'Dallas', 837, 75219, 'TX', False, 'condo', 1998, False, True, False, False, 2.6, 0.6),
    ('Active', 2.0, 'Dallas', 2785, 75219, 'TX', False, 'condo', 2015, False, True, False, True, 2.6, 0.7)
]

columns = ['status', 'baths', 'city', 'sqft', 'zipcode', 'state', 'pool_encoded', 'Type', 'Year built', 'Heating_encoded', 'Cooling_encoded', 'Parking_encoded', 'fireplace_encoded', 'school_rating _mean', 'school_dist_min']

df_test = pd.DataFrame(data, columns=columns)

df_test.head()

,status,baths,city,sqft,zipcode,state,pool_encoded,Type,Year built,Heating_encoded,Cooling_encoded,Parking_encoded,fireplace_encoded,school_rating _mean,school_dist_min
0,Active,3.0,Washington,801,20004,DC,False,other,1991,False,False,False,False,6.0,0.2
1,Active,2.0,Dallas,832,75219,TX,False,condo,1998,False,True,False,False,2.6,0.6
2,Active,4.0,Dallas,2102,75219,TX,False,condo,2019,False,True,False,False,2.0,0.1
3,Active,2.0,Dallas,837,75219,TX,False,condo,1998,False,True,False,False,2.6,0.6
4,Active,2.0,Dallas,2785,75219,TX,False,condo,2015,False,True,False,True,2.6,0.7


In [64]:
# функция принимает датасет и модель, затем обрабатывет 
# датасет и передает его в модель для получения предсказания, предсказание выводится в нормальном виде
def preprocess_and_predict(df_input, model):
    
    def log_data(df_input):
        df_output = df_input.copy()
        df_output['zipcode'] = df_output['zipcode'].astype(str)
        df_output['Year built'] = df_output['Year built'].astype(str)
        
        #scaler = MinMaxScaler()
        for column in ['baths', 'sqft', 'school_rating _mean', 'school_dist_min']:
            #df_output[column] = scaler.fit_transform(df_output[[column]])[:,0]
            df_output[column] = df_output[column].apply(lambda x: abs(x))
            constant = 1e-6
            df_output[column] = np.log(df_output[column] + constant)
        return df_output

    X_test = log_data(df_input)
    y_test_pred_loaded = model.predict(X_test)
    target = np.exp(y_test_pred_loaded)
    rounded_target = np.round(target)
    print(rounded_target)
    
    return rounded_target

In [ ]:
predictions = preprocess_and_predict(df_test, loaded_model)
# [506389. 224225. 551250. 224081. 626743.]

### Контейнеризация Docker

Образ создаем в терминале ПК командой:

docker build -t diplom .

Ждем окончания установки всех необходимых библиотек и запускаем наш сервер командой

docker run -p 5000:5000 diplom python server.py


после запуска страница браузера по адресу 127.0.0.1:5000 выдаст сообщение "Тестовое сообщение. Сервер запущен!"

далее нужно перейти в терминале в директорию test

cd test
и запустить скрипт отправки запроса client.py

python3 client.py

# Заключение

Обучена модель по поставленной задаче предсказания цены нового объекта недвижимости.